# Lab 9

— Generative Adversarial Networks (GANs)

Build and train a **vanilla GAN** using the code stored in the following file. Be aware, the code you write here and its instructions is not going to be the same expected in the file below.

- `10-generative_adversarial_network.ipynb`
  and concepts from the slides: `9 - GANs.pptx`.

Each task below includes **explicit steps**, **where to find matching content** in the script, and the **expected output**.

\*\* A note about architectures
If the neural network architecture isn't clearly specified, you are given freedom to experiment. Do not take my output results as the "solution" to try to achieve but strive for what I have or even better! But even if you are 5% off and lower, that doesn't mean you have done it wrong, you have done it differently. As long as the specified parameters required in the instructions are present, you will get full points.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/username/repo/blob/main/path-to-file)  
**Students:** Replace `username`, `repo`, and `path-to-file` with your own GitHub username, repository name, and the path to this file.  
After opening in Colab, go to **File → Save a copy to GitHub** (your repo) before editing.

#### NOTE: You will need to use a GPU on Google Colab for this to run in a reasonable amount of time without memory running out. I suggest using A100 GPU. Edit -> Notebook Settings


## H.1 Data Preparation — MNIST 28×28 Grayscale

**Reference script:** `10-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Imports and dataset load for MNIST not in reference file.
- Reshaping done in In [51]. May require additional steps.


In [ ]:
# TODO (H.1): Load MNIST, scale to [0,1], and reshape to (N,28,28,1).
# Steps:
# 1) Use keras.datasets.mnist.load_data()
# 2) Convert to float32 and divide by 255.0
# 3) Expand dimensions to add channel axis
# Output variables: x_train (float32, shape=(60000,28,28,1))
pass


**Expected output (H.1):**

- `x_train.shape == (60000, 28, 28, 1)`
- `x_train.min() >= 0.0` and `x_train.max() <= 1.0`


## H.2 Define the Discriminator

**Reference script:** `10-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Code in the file uses a special model building class
- Refer to previous scripts for building a Sequential
- as a hint here is a start: model = keras.Sequential([
- Follow instructions below
- Optimizer setup for discriminator (often Adam with β1≈0.5).


In [ ]:
# TODO (H.2): Build the discriminator.
# Architecture (match script style):
#   Input: (28,28,1)
#   Conv2D(64, kernel_size=5, strides=2, padding='same') → LeakyReLU(0.2) → Dropout(0.3)
#   Conv2D(128, kernel_size=5, strides=2, padding='same') → LeakyReLU(0.2) → Dropout(0.3)
#   Flatten → Dense(1, activation='sigmoid')
# Compile with loss='binary_crossentropy', optimizer=Adam(lr ~ 2e-4, beta_1 ~ 0.5), metrics=['accuracy']
# Name your model variable: discriminator
# Print the model summary: discriminator.summary()
pass


**Expected output (H.2):**

- `discriminator` is a compiled Keras model.
- Calling `discriminator(tf.zeros([1,28,28,1]))` returns a (1,1) tensor in [0,1].


## H.3 Define the Generator

**Reference script:** `10-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Again code in that script create from a special model building function
- Create yours as we have been doing using Sequential
- Hint: model = keras.Sequential([
- Latent vector size (e.g., `latent_dim = 100`).


In [ ]:
# TODO (H.3): Build the generator that maps z∼N(0,1) of shape (latent_dim,) to (28,28,1).
# Suggested pattern (match script):
#   Dense(7*7*128) → LeakyReLU → Reshape(7,7,128)
#   Conv2DTranspose(64, kernel_size=5, strides=2, padding='same') → LeakyReLU
#   Conv2DTranspose(1,  kernel_size=5, strides=2, padding='same', activation='sigmoid')
# Name your model variable: generator
# Print the model summary: generator.summary()
pass


**Expected output (H.3):**

- `generator` is a Keras model that, given a (batch, latent_dim) noise input, outputs images of shape (batch,28,28,1) in [0,1].


## H.4 Adversarial (GAN) Model

**Reference script:** `10-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Follow along with instructions below. Adversarial building is different in the 10-...script


In [ ]:
# TODO (H.4): Build the GAN by stacking generator → discriminator.
# Steps:
# 1) Set discriminator.trainable = False (for the GAN compile step)
# 2) Create an Input for z (latent_dim,)
# Consider keras.Input for this
# 3) Pass through generator then discriminator
# Hint will look something like this
# img = generator(z_in)
# score = discriminator(img)
# 4) Compile GAN with loss='binary_crossentropy' and Adam(lr ~ 2e-4, beta_1 ~ 0.5)
# Name your model variable: gan
# Print the model summary: gan.summary()
pass


**Expected output (H.4):**

- `gan` is a compiled model that takes noise and returns a real/fake score.


## H.5 Training Loop (Memory-aware)

**Reference script:** `10-generative_adversarial_network.ipynb`


**Where to find in the script:**

- The custom loop alternating discriminator and generator updates (often using real labels=1 and fake labels=0).
- Epoch logging and periodic sampling of generated images.


In [ ]:
# TODO: Implement the full GAN training loop.
#
# Use the reference training function provided in the lab description as a GUIDE.
# Your loop here should follow the same conceptual steps even though the exact
# shapes, labels, and method calls may differ depending on your generator,
# discriminator, and GAN models.
#
# -------------------------------------------------------------------------
# The reference function illustrates the following key ideas:
# 
# 1. SAMPLE REAL IMAGES:
#    - Pull a batch of real images from your training data.
#    - Reshape them if needed to (batch, height, width, channels).
#
# 2. GENERATE FAKE IMAGES:
#    - Sample random noise from a uniform or normal distribution.
#    - Pass this noise through the generator to create fake images.
#
# 3. PREPARE DISCRIMINATOR INPUTS:
#    - Combine real and fake images into a single batch for training.
#    - Create labels:
#          real images → label 1
#          fake images → label 0
#
# 4. TRAIN THE DISCRIMINATOR:
#    - Train on real + fake inputs in one step OR train separately on
#      real images and fake images (as shown in the more modern GAN loop).
#    - Track the discriminator's loss and accuracy.
#
# 5. TRAIN THE GENERATOR THROUGH THE GAN MODEL:
#    - Sample a new batch of random noise.
#    - Assign labels of 1 (meaning “real”) to that noise batch.
#      Why? Because the generator wants to fool the discriminator.
#    - Freeze the discriminator’s weights so that GAN training updates ONLY
#      the generator.
#
# 6. LOG METRICS:
#    - Track generator losses and discriminator losses every epoch.
#    - Print summary statistics periodically (e.g., every 5 epochs).
#
# 7. VISUALIZE GENERATED IMAGES:
#    - Use a function like sample_grid() to visualize progress.
#    - Always generate from a FIXED noise vector so that improvements
#      over epochs are easy to see.
#
# -------------------------------------------------------------------------
# HOW TO MAP THE REFERENCE FUNCTION TO YOUR TRAINING LOOP HERE:
#
# In the reference code:
#     real_imgs = <sampled real batch>
#     fake_imgs = generator.predict(...)
#     d_metrics = discriminator.train_on_batch(...)
#     a_metrics = adversarial_model.train_on_batch(...)
#
# In your loop for this lab, use:
#     x_real  = sampled real images
#     x_fake  = generator(z)
#     discriminator.train_on_batch(...)
#     gan.train_on_batch(...)
#
# The IDEA is the same:
#   - Discriminator gets real+fake with 1s and 0s.
#   - Generator gets noise but uses label=1 to push it toward realism.
#
# -------------------------------------------------------------------------
# STRUCTURE TO FOLLOW:
#
# for epoch in range(1, EPOCHS+1):
#     Initialize lists for tracking losses
#
#     for step in range(steps_per_epoch):
#         -----------------------------
#         (A) Train Discriminator
#         -----------------------------
#         - Sample real images
#         - Generate fake images
#         - Train discriminator on both real and fake batches
#           (you may call train_on_batch twice)
#
#         -----------------------------
#         (B) Train Generator (via GAN)
#         -----------------------------
#         - Sample new random noise
#         - Train GAN with label=1
#           (this updates ONLY the generator)
#
#     Print epoch metrics
#
#     Every N epochs (e.g., 5):
#         sample_grid(generator, fixed_noise)
#
# -------------------------------------------------------------------------
# IMPORTANT:
# - The discriminator alternates between training on real and fake images.
# - The generator learns by trying to fool the discriminator.
# - Freezing/unfreezing discriminator.trainable at the right times matters.
# - Even though the TRAINING STYLE is similar to the reference function,
#   use the architecture and dataset defined in THIS lab.
#
# -------------------------------------------------------------------------
# Implement your full training loop below following this structure.
pass


**Expected output (H.5):**

- Per-epoch logs: D loss/acc and GAN (generator) loss.
- A few image grids showing samples improving over time.
- Final print of losses across epochs.


## Discussion Questions


1. Why can GAN training be unstable? Name two symptoms and one stabilization trick.
2. What happens if the discriminator is much stronger than the generator? How would you adjust training?
3. If generated digits are blurry, which change to the generator would you try first, and why?
